In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/watermarked-and-unwatermarked-text-truncated/data_trunk.csv


In [2]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 15.6 MB/s eta 0:00:00


In [3]:
import json

import torch
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessorList
from transformers import MarianMTModel, MarianTokenizer

from tqdm import tqdm

from collections import defaultdict

import numpy as np
import scipy


In [4]:
translation_language = "french"
device = "cuda" if torch.cuda.is_available() else "cpu"
if translation_language == "french":
    en_ne_model_name = "Helsinki-NLP/opus-mt-tc-big-en-fr"
    en_ne_tokenizer = MarianTokenizer.from_pretrained(en_ne_model_name)
    en_ne_model = MarianMTModel.from_pretrained(en_ne_model_name).to(device)

    ne_en_model_name = "Helsinki-NLP/opus-mt-tc-big-fr-en"
    ne_en_tokenizer = MarianTokenizer.from_pretrained(ne_en_model_name)
    ne_en_model = MarianMTModel.from_pretrained(ne_en_model_name).to(device)
elif translation_language == "russian":
    en_ne_model_name = "Helsinki-NLP/opus-mt-en-ru"
    en_ne_tokenizer = MarianTokenizer.from_pretrained(en_ne_model_name)
    en_ne_model = MarianMTModel.from_pretrained(en_ne_model_name).to(device)

    ne_en_model_name = "Helsinki-NLP/opus-mt-ru-en"
    ne_en_tokenizer = MarianTokenizer.from_pretrained(ne_en_model_name)
    ne_en_model = MarianMTModel.from_pretrained(ne_en_model_name).to(device)
else:
    raise


tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/820k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/461M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [5]:
def rt_translate(text):
    try:
        tokens = en_ne_tokenizer(text.split('. '), return_tensors="pt", padding=True).to(device)
        tokens = en_ne_model.generate(**tokens,max_new_tokens=52)
        french_text = ' '.join([en_ne_tokenizer.decode(t, skip_special_tokens=True) for t in tokens])

        tokens = ne_en_tokenizer(french_text.split('. '), return_tensors="pt", padding=True).to(device)
        tokens = ne_en_model.generate(**tokens,max_new_tokens=512)
        roundtrip_text = ' '.join([ne_en_tokenizer.decode(t, skip_special_tokens=True) for t in tokens])
    except:
        roundtrip_text = ""
    return roundtrip_text

In [6]:
# Read data from CSV
data = pd.read_csv("/kaggle/input/watermarked-and-unwatermarked-text-truncated/data_trunk.csv")
# print(data)

# Filter for watermarked "Generated text" with a limit of 1000 rows
filtered_data = data[data["label"] == "watermarked"][["Generated Text"]]
# Initialize empty list to store paraphrased text
paraphrased_text = []

# Loop through the data using tqdm for progress bar
for text in tqdm(filtered_data["Generated Text"].tolist()):
    # Perform round-trip translation and append to list
    translated = rt_translate(text)
#     print(translated)
    paraphrased_text.append(translated)

# Create a new dataframe with original and paraphrased text
df = pd.DataFrame({"Original Text": filtered_data["Generated Text"], "Paraphrased Text": paraphrased_text})

# Save dataframe as CSV file
df.to_csv("paraphrased_data_fr.csv", index=False)

print("Paraphrasing complete! Paraphrased data saved to paraphrased_data_fr.csv")

100%|██████████| 1000/1000 [13:23<00:00,  1.24it/s]

Paraphrasing complete! Paraphrased data saved to paraphrased_data_fr.csv


In [7]:
translation_language = "russian"
device = "cuda" if torch.cuda.is_available() else "cpu"
if translation_language == "french":
    en_ne_model_name = "Helsinki-NLP/opus-mt-tc-big-en-fr"
    en_ne_tokenizer = MarianTokenizer.from_pretrained(en_ne_model_name)
    en_ne_model = MarianMTModel.from_pretrained(en_ne_model_name).to(device)

    ne_en_model_name = "Helsinki-NLP/opus-mt-tc-big-fr-en"
    ne_en_tokenizer = MarianTokenizer.from_pretrained(ne_en_model_name)
    ne_en_model = MarianMTModel.from_pretrained(ne_en_model_name).to(device)
elif translation_language == "russian":
    en_ne_model_name = "Helsinki-NLP/opus-mt-en-ru"
    en_ne_tokenizer = MarianTokenizer.from_pretrained(en_ne_model_name)
    en_ne_model = MarianMTModel.from_pretrained(en_ne_model_name).to(device)

    ne_en_model_name = "Helsinki-NLP/opus-mt-ru-en"
    ne_en_tokenizer = MarianTokenizer.from_pretrained(ne_en_model_name)
    ne_en_model = MarianMTModel.from_pretrained(ne_en_model_name).to(device)
else:
    raise


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

...

target.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [8]:
# Read data from CSV
data = pd.read_csv("/kaggle/input/watermarked-and-unwatermarked-text-truncated/data_trunk.csv")
# print(data)

# Filter for watermarked "Generated text" with a limit of 1000 rows
filtered_data = data[data["label"] == "watermarked"][["Generated Text"]]
# Initialize empty list to store paraphrased text
paraphrased_text = []

# Loop through the data using tqdm for progress bar
for text in tqdm(filtered_data["Generated Text"].tolist()):
    # Perform round-trip translation and append to list
    translated = rt_translate(text)
#     print(translated)
    paraphrased_text.append(translated)

# Create a new dataframe with original and paraphrased text
df = pd.DataFrame({"Original Text": filtered_data["Generated Text"], "Paraphrased Text": paraphrased_text})

# Save dataframe as CSV file
df.to_csv("paraphrased_data_russian.csv", index=False)

print("Paraphrasing complete! Paraphrased data saved to paraphrased_data_russian.csv")

100%|██████████| 1000/1000 [12:22<00:00,  1.35it/s]

Paraphrasing complete! Paraphrased data saved to paraphrased_data_russian.csv
